In [1]:
import os
from PIL import Image
import tensorflow as tf

In [2]:
train_covid_dir = os.path.join('/dataset/train/covid19')

train_normal_dir = os.path.join('/dataset/train/normal')

test_covid_dir = os.path.join('/dataset/test/covid19')

test_normal_dir = os.path.join('/dataset/test/normal')

In [3]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(250, 250, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 248, 248, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 124, 124, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 61, 61, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 59, 59, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 27, 27, 64)        3

In [5]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255, rotation_range=50,
 featurewise_center = True,
 featurewise_std_normalization = True,
 width_shift_range=0.2,
 height_shift_range=0.2,
 shear_range=0.25,
 zoom_range=0.1,
 zca_whitening = True,
 channel_shift_range = 20,
 horizontal_flip = True ,
 vertical_flip = True ,
 validation_split = 0.2,
 fill_mode='constant')

validation_datagen = ImageDataGenerator(rescale=1/255, rotation_range=50,
 featurewise_center = True,
 featurewise_std_normalization = True,
 width_shift_range=0.2,
 height_shift_range=0.2,
 shear_range=0.25,
 zoom_range=0.1,
 zca_whitening = True,
 channel_shift_range = 20,
 horizontal_flip = True ,
 vertical_flip = True ,
 validation_split = 0.2,
 fill_mode='constant')

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        'dataset/train/',  # This is the source directory for training images
        target_size=(250, 250),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        'dataset/test/',  # This is the source directory for training images
        target_size=(250, 250),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 656 images belonging to 2 classes.
Found 157 images belonging to 2 classes.


/home/rootuser/project/lung-xray/venv/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:342: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn('This ImageDataGenerator specifies '


In [8]:
history = model.fit(
      train_generator,
      steps_per_epoch=5,  
      epochs=80,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=4)

Epoch 1/80
5/5 [==============================] - 20s 4s/step - loss: 0.7743 - accuracy: 0.5189 - val_loss: 0.6916 - val_accuracy: 0.5000
Epoch 2/80
5/5 [==============================] - 19s 4s/step - loss: 0.6925 - accuracy: 0.4811 - val_loss: 0.6865 - val_accuracy: 0.4688
Epoch 3/80
5/5 [==============================] - 20s 4s/step - loss: 0.7328 - accuracy: 0.5379 - val_loss: 0.6826 - val_accuracy: 0.6172
Epoch 4/80
5/5 [==============================] - 18s 4s/step - loss: 0.6805 - accuracy: 0.6042 - val_loss: 0.6653 - val_accuracy: 0.6016
Epoch 5/80
5/5 [==============================] - 22s 4s/step - loss: 0.6425 - accuracy: 0.6023 - val_loss: 0.6804 - val_accuracy: 0.5703
Epoch 6/80
5/5 [==============================] - 18s 4s/step - loss: 0.6729 - accuracy: 0.5777 - val_loss: 0.6301 - val_accuracy: 0.6484
Epoch 7/80
5/5 [==============================] - 19s 4s/step - loss: 0.6436 - accuracy: 0.6042 - val_loss: 0.6189 - val_accuracy: 0.6562
Epoch 8/80
5/5 [==================

5/5 [==============================] - 19s 4s/step - loss: 0.2141 - accuracy: 0.9129 - val_loss: 0.1738 - val_accuracy: 0.9219
Epoch 61/80
5/5 [==============================] - 18s 4s/step - loss: 0.1672 - accuracy: 0.9432 - val_loss: 0.1802 - val_accuracy: 0.9531
Epoch 62/80
5/5 [==============================] - 19s 4s/step - loss: 0.1809 - accuracy: 0.9489 - val_loss: 0.1989 - val_accuracy: 0.9375
Epoch 63/80
5/5 [==============================] - 23s 5s/step - loss: 0.1624 - accuracy: 0.9413 - val_loss: 0.1513 - val_accuracy: 0.9531
Epoch 64/80
5/5 [==============================] - 17s 3s/step - loss: 0.1456 - accuracy: 0.9545 - val_loss: 0.1813 - val_accuracy: 0.9141
Epoch 65/80
5/5 [==============================] - 21s 4s/step - loss: 0.2637 - accuracy: 0.8813 - val_loss: 0.1711 - val_accuracy: 0.9297
Epoch 66/80
5/5 [==============================] - 19s 4s/step - loss: 0.2163 - accuracy: 0.9110 - val_loss: 0.3025 - val_accuracy: 0.8906
Epoch 67/80
5/5 [======================

In [9]:
import numpy as np
from keras.preprocessing import image




path = 'covid19-2.jpeg'
img = image.load_img(path, target_size=(250, 250))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes)
if classes[0]>0.5:
    print(" is normal")
else:
    print("has coronavirus")


Using TensorFlow backend.


[[0.]]
has coronavirus


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
#visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)
# Let's prepare a random input image from the training set.
horse_img_files = [os.path.join(train_covid_dir, f) for f in train_horse_names]
human_img_files = [os.path.join(train_covid_dir, f) for f in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)

img = load_img(img_path, target_size=(150, 150))  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    if len(feature_map.shape) == 4:
        # Just do this for the conv / maxpool layers, not the fully-connected layers
        n_features = feature_map.shape[-1]  # number of features in feature map
        # The feature map has shape (1, size, size, n_features)
        size = feature_map.shape[1]
        # We will tile our images in this matrix
        display_grid = np.zeros((size, size * n_features))
        for i in range(n_features):
          # Postprocess the feature to make it visually palatable
          x = feature_map[0, :, :, i]
          x -= x.mean()
          x /= x.std()
          x *= 64
          x += 128
          x = np.clip(x, 0, 255).astype('uint8')
          # We'll tile each filter into this big horizontal grid
          display_grid[:, i * size : (i + 1) * size] = x
        # Display the grid
        scale = 20. / n_features
        plt.figure(figsize=(scale * n_features, scale))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [59]:
classes[0]

array([1.], dtype=float32)

In [10]:
model.save('covid_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: covid_model/assets


In [11]:
reconstructed_model = tf.keras.models.load_model('covid_model')

In [13]:
path = 'NORMAL2-IM-0285-0001.jpeg'
img = image.load_img(path, target_size=(250, 250))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = reconstructed_model.predict(images, batch_size=10)
print(classes[0])
if classes[0]>0.5:
    print(" is normal")
else:
    print("has coronavirus")

[1.]
 is normal
